In [ ]:
# ----------------------------------------------------------------------------
# Title: Assignment 8.2
# Author: Surenther Selvaraj
# Date: 01 Nov 2025
# Modified By: Surenther Selvaraj
# Description: Term Project Milestone 2: Data Preparation
# Data: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis
# ----------------------------------------------------------------------------

## Term Project Milestone 2: Data Preparation Plan
This document outlines the initial data preparation and feature engineering strategy for the classification model, specifically focusing on identifying and dropping features that are non-predictive, redundant, or problematic for the modeling process.

### 1. Feature Drop Strategy

To optimize the model's performance and prevent data leakage or multicollinearity, several features from the raw dataset will be excluded.

| Feature to Drop  | Rationale for Exclusion  |  Category |
|---|---|---|
| ID  | This is a unique customer identifier. It has no predictive power and should be dropped to prevent the model from memorizing individual observations.  | Non-Predictive  |
| Dt_Customer  | Date of customer enrollment. While conversion time might be relevant, the more actionable and direct time-based feature, Recency, is already available and highly predictive (as shown in Graph 3).  | Redundant/High Cardinality  |
| Z_CostContact  | Standardized cost of customer contact. This column is constant across all customers in the dataset (a value of 3) and therefore provides zero variance or predictive information.  | Zero-Variance |
| Z_Revenue | Standardized revenue from contact. This column is constant across all customers (a value of 11) and provides zero variance or predictive information.  | Zero-Variance |
| AcceptedCmp1 to AcceptedCmp5 | Response to previous campaigns. These features represent the outcome of past offers. Using them to predict the current campaign's Response (which is effectively AcceptedCmp6) creates severe data leakage, as a positive response to a previous similar campaign is an unnaturally strong predictor for the current one. Dropping these ensures the model generalizes to new campaigns.  | Data Leakage |
| Complain | Customer complaint status. While technically a potential predictor, the count of complaints is extremely low, leading to highly skewed classes that provide minimal signal for a classification model focused on positive response prediction.  | Near Zero-Variance |


### 2. Features to Keep and Transform

The remaining features will be kept as they are highly relevant to the business problem (customer wealth, engagement, and past purchase behavior).

| Feature Type  | Features to Keep |  Required Transformation |
|---|---|---|
| Target  | Response  | None (already binary: 0 or 1)  |
| Demographic  | Income, Year_Birth (used to calculate Age), Marital_Status, Education, Kidhome, Teenhome  | Age is derived. Marital_Status and Education require one-hot encoding.  |
| Behavior  | Recency | None (numerical, already normalized).  |
| Spending  | MntWines, MntFruits, MntMeatProducts, MntFishProducts, MntSweetProducts, MntGoldProds | None (numerical, may require scaling).  |
| Channel  | NumDealsPurchases, NumWebPurchases, NumCatalogPurchases, NumStorePurchases, NumWebVisitsMonth | None (numerical, may require scaling).  |

### 1. Loading Data/Extraction

This initial step is responsible for importing the necessary pandas library and defining the file path and feature lists required for the project. It explicitly defines the CHANNEL_FEATURES (e.g., NumWebPurchases) and other CORE_FEATURES (e.g., Income, Response) that are central to the customer campaign response model. The code attempts to load the marketing_campaign.csv file, and includes error handling to notify the user if the data file cannot be located. Successfully loading the data sets the stage for all subsequent cleaning and preparation activities.

In [19]:
# Python Data Extraction for Channel Features
import pandas as pd

# Define the file path (Assuming the file is in the same directory)
FILE_PATH = 'marketing_campaign.csv'

# Define the list of channel features selected from the Milestone 2 plan
CHANNEL_FEATURES = [
    'NumDealsPurchases',
    'NumWebPurchases',
    'NumCatalogPurchases',
    'NumStorePurchases',
    'NumWebVisitsMonth'
]

# Add other core features necessary for context (e.g., Target, Income)
# NOTE: We temporarily include all spending features to create the useful engineered feature below.
CORE_FEATURES = ['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Response', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']

# --- Load Data ---
try:
    df = pd.read_csv(FILE_PATH, sep='\t')
    print(f"Successfully loaded {len(df)} rows from {FILE_PATH}.")
except FileNotFoundError:
    print(f"Error: File not found at {FILE_PATH}. Please check the path and filename.")
    exit()

Successfully loaded 2240 rows from marketing_campaign.csv.


### 2. Deal with missing data
This crucial step addresses the small number of missing values (24 rows, or 1.07%) found in the highly critical Income predictor variable. Rather than dropping these few rows, which is often acceptable but can sometimes discard valuable information, the code implements median imputation. Using the median is preferred because Income is known to be heavily skewed and contain outliers, ensuring the imputed values do not unduly distort the feature's statistical distribution. This method maintains the full dataset size while creating a complete column ready for transformation.

In [20]:
from sklearn.impute import SimpleImputer
# --- Data Cleaning (Handling Missing Values) ---

# Rationale: 'Income' has 24 missing values (approx. 1.07%). 
# Because Income is highly skewed (as seen in EDA), median imputation is preferred over dropping 
# rows or using the mean, as it minimizes distortion to the distribution.

imputer = SimpleImputer(strategy='median')
df['Income'] = imputer.fit_transform(df[['Income']])

# Check the dataset size after imputation (no rows dropped)
print(f"Imputed missing 'Income' values using the median. Total rows: {len(df)}")

Imputed missing 'Income' values using the median. Total rows: 2240


### 3. Feature Selection
This step focuses on creating a clean, actionable subset of the data by explicitly selecting and combining the core features identified as most relevant for the classification model. It merges the categorical, numerical, and target variables (including all spending features needed for engineering) into the df_extracted DataFrame. Within this step, a preliminary engineered feature, Total_Purchases_Count, is calculated by summing the three main purchase channel counts. This count gives an initial, unscaled metric of overall customer engagement for immediate assessment.

In [21]:
# --- Feature Selection ---
# Combine the core features and the specific channel features selected.
features_to_extract = [col for col in CORE_FEATURES + CHANNEL_FEATURES if col in df.columns]

# Select only the required columns
df_extracted = df[features_to_extract].copy()

# --- Initial Assessment of Channel Purchasing Habits ---
df_extracted['Total_Purchases_Count'] = (
    df_extracted['NumWebPurchases'] + 
    df_extracted['NumCatalogPurchases'] + 
    df_extracted['NumStorePurchases']
)

print("\n--- Total Purchases Count Summary ---")
print(df_extracted['Total_Purchases_Count'].describe())

print("\nData extraction and selection for channel features complete.")



--- Total Purchases Count Summary ---
count    2240.000000
mean       12.537054
std         7.205741
min         0.000000
25%         6.000000
50%        12.000000
75%        18.000000
max        32.000000
Name: Total_Purchases_Count, dtype: float64

Data extraction and selection for channel features complete.


### 4. Feature Transformation
This step implements MinMaxScaler on a selection of numerical features, including Income and the original channel count variables. The purpose of this transformation is to normalize the scale of these predictors to a uniform range between 0 and 1. Normalization is essential because it prevents variables with inherently large raw values, such as Income, from numerically dominating the model's objective function during training. Finally, the script demonstrates re-creating a purchase count feature using the new scaled channel variables for assessment.

In [22]:
from sklearn.preprocessing import MinMaxScaler

# --- Feature Transformation (Scaling Numerical Data) ---
# Define all numerical features that need scaling, including Income
SCALING_FEATURES = ['Income'] + CHANNEL_FEATURES

# Initialize the Scaler (MinMaxScaler is robust for skewed count/revenue data)
scaler = MinMaxScaler()

# Apply the scaling transformation in place
df_extracted[SCALING_FEATURES] = scaler.fit_transform(df_extracted[SCALING_FEATURES])
print("\nScaling applied successfully to Income and Channel Features.")


# Example: Re-create the new feature using scaled values
df_extracted['Total_Purchases_Count_Scaled'] = (
    df_extracted['NumWebPurchases'] + 
    df_extracted['NumCatalogPurchases'] + 
    df_extracted['NumStorePurchases']
)

print("\n--- Total Scaled Purchases Count Summary ---")
print(df_extracted['Total_Purchases_Count_Scaled'].describe())

print("\nData scaling complete.")


Scaling applied successfully to Income and Channel Features.

--- Total Scaled Purchases Count Summary ---
count    2240.000000
mean        0.691761
std         0.380945
min         0.000000
25%         0.340558
50%         0.646825
75%         0.987815
max         1.651913
Name: Total_Purchases_Count_Scaled, dtype: float64

Data scaling complete.


### 5. Engineer new useful features.
This step focuses on creating highly predictive behavioral and monetary features from the raw data. It calculates Total_Purchases_Count as a simple measure of engagement across channels to assess customer activity. Crucially, it creates Total_Spent, which acts as a robust proxy for customer lifetime value (CLV) and will be a strong model predictor. Additionally, the Prop_Online_Purchases ratio is calculated to capture a customer's channel preference (online vs. physical store). These new features capture deeper insight into customer wealth and behavioral habits than the original columns alone.

In [ ]:
# Calculate Proportion of Online Purchases (Web + Catalog Purchases / Total Purchases)
df_extracted['Prop_Online_Purchases'] = (
    df_extracted['NumWebPurchases'] + df_extracted['NumCatalogPurchases']
) / df_extracted['Total_Purchases_Count'].replace(0, 1) # Replace 0 with 1 temporarily to avoid zero division
df_extracted.loc[df_extracted['Total_Purchases_Count'] == 0, 'Prop_Online_Purchases'] = 0

# Calculate Total Spending (Total Monetary Value)
df_extracted['Total_Spent'] = (
    df_extracted['MntWines'] + df_extracted['MntFruits'] + df_extracted['MntMeatProducts'] + 
    df_extracted['MntFishProducts'] + df_extracted['MntSweetProducts'] + df_extracted['MntGoldProds']
)

print("\nEngineered features (Total_Purchases_Count, Prop_Online_Purchases, Total_Spent) created successfully.")


Engineered features (Total_Purchases_Count, Prop_Online_Purchases, Total_Spent) created successfully.
Scaling applied successfully to Income, Spending, and Channel Features.

--- Statistical Summary of Key Engineered Features ---
                        count      mean       std  min       25%       50%  \
Total_Spent            2240.0  0.238412  0.238988  0.0  0.025298  0.155159   
Total_Purchases_Count  2240.0  0.391783  0.225179  0.0  0.187500  0.375000   
Prop_Online_Purchases  2240.0  0.018055  0.005491  0.0  0.014815  0.018330   

                            75%       max  
Total_Spent            0.412897  1.000000  
Total_Purchases_Count  0.562500  1.000000  
Prop_Online_Purchases  0.021693  0.037037  


### 6. Create Dummy Variables
This mandatory step converts the two remaining categorical text features (Education and Marital_Status) into a numerical format suitable for machine learning models. Before conversion, the Marital_Status column is cleaned by grouping rare or similar labels (like 'Divorced', 'Widow', and 'Alone') into the unified 'Single' category to reduce noise and simplify the model. The features are then converted using one-hot encoding (pd.get_dummies), and one column is dropped per feature (drop_first=True) to prevent multicollinearity.

In [24]:
# --- Create Dummy Variables (One-Hot Encoding) ---

# Clean Marital Status: Combine small/similar groups for simplicity
df_extracted['Marital_Status'] = df_extracted['Marital_Status'].replace(['Divorced', 'Widow', 'Alone', 'YOLO', 'Absurd'], 'Single')

# Create Dummy Variables for Categorical Features
df_extracted = pd.get_dummies(df_extracted, columns=['Education', 'Marital_Status'], drop_first=True)
print("Dummy variables created for 'Education' and 'Marital_Status'.")


Dummy variables created for 'Education' and 'Marital_Status'.


### 7. Feature Transformation
This final preparation step applies MinMaxScaler to the core numerical features, including the newly engineered Total_Spent and Total_Purchases_Count. Normalizing these predictors ensures that features with vastly different scales—like Income and the low-count channel visits—contribute fairly to the model training process. Scaling prevents features with larger magnitudes from dominating the loss function, which is critical for optimization. The process is validated by inspecting the descriptive statistics of the scaled engineered features, confirming they are ready for modeling.

In [25]:
# --- Feature Transformation (Scaling Numerical Data) ---

# Define all numerical features that need scaling, including new engineered ones
SCALING_FEATURES = ['Income'] + CHANNEL_FEATURES + ['Total_Purchases_Count', 'Total_Spent', 'Prop_Online_Purchases']
SCALING_NUMERICALS = ['Income', 'Total_Spent'] + CHANNEL_FEATURES + ['Total_Purchases_Count']

# Initialize the Scaler (MinMaxScaler is robust for skewed count/revenue data)
scaler = MinMaxScaler()

# Apply the scaling transformation in place
df_extracted[SCALING_NUMERICALS] = scaler.fit_transform(df_extracted[SCALING_NUMERICALS])
print("Scaling applied successfully to Income, Spending, and Channel Features.")

# --- Inspection of Final Transformed Data ---

print("\n--- Statistical Summary of Key Engineered Features ---")
print(df_extracted[['Total_Spent', 'Total_Purchases_Count', 'Prop_Online_Purchases']].describe().transpose())

Scaling applied successfully to Income, Spending, and Channel Features.

--- Statistical Summary of Key Engineered Features ---
                        count      mean       std  min       25%       50%  \
Total_Spent            2240.0  0.238412  0.238988  0.0  0.025298  0.155159   
Total_Purchases_Count  2240.0  0.391783  0.225179  0.0  0.187500  0.375000   
Prop_Online_Purchases  2240.0  0.018055  0.005491  0.0  0.014815  0.018330   

                            75%       max  
Total_Spent            0.412897  1.000000  
Total_Purchases_Count  0.562500  1.000000  
Prop_Online_Purchases  0.021693  0.037037  
